In [3]:
from src.get_data import read_params
import argparse
import mlflow
from mlflow.tracking import MlflowClient
from pprint import pprint
import joblib
import os


def log_production_model(config_path):
    config = read_params(config_path)

    mlflow_config = config["mlflow_config"]
    model_name = mlflow_config["registered_model_name"]
    remote_server_uri = mlflow_config["remote_server_uri"]
    mlflow.set_tracking_uri(remote_server_uri)

    runs = mlflow.search_runs(["1"])
    lowest = runs["metrics.mae"].sort_values(ascending=True)[0]
    lowest_run_id = runs[runs["metrics.mae"] == lowest]["run_id"][0]

    client = MlflowClient()
    for mv in client.search_model_versions(f"name='{model_name}'"):
        mv = dict(mv)
        if mv["run_id"] == lowest_run_id:
            current_version = mv["version"]
            logged_model = mv["source"]
            pprint(mv, indent=4)
            client.transition_model_version_stage(
                name=model_name,
                version=current_version,
                stage="Production"
            )
        else:
            current_version = mv["version"]
            logged_model = mv["source"]
            pprint(mv, indent=4)
            client.transition_model_version_stage(
                name=model_name,
                version=current_version,
                stage="Staging"
            )
log_production_model("../params.yaml")

{   'creation_timestamp': 1666855133357,
    'current_stage': 'Staging',
    'description': '',
    'last_updated_timestamp': 1666938780328,
    'name': 'ElasticNetWineModel',
    'run_id': 'ac8a96765d2241149dcd9432a201b5e2',
    'run_link': '',
    'source': './artifacts/1/ac8a96765d2241149dcd9432a201b5e2/artifacts/model',
    'status': 'READY',
    'status_message': '',
    'tags': {},
    'user_id': '',
    'version': '1'}
{   'creation_timestamp': 1666855279830,
    'current_stage': 'Staging',
    'description': '',
    'last_updated_timestamp': 1666938780298,
    'name': 'ElasticNetWineModel',
    'run_id': '49bae4a0c90540d9aac59e04dcd5b45f',
    'run_link': '',
    'source': './artifacts/1/49bae4a0c90540d9aac59e04dcd5b45f/artifacts/model',
    'status': 'READY',
    'status_message': '',
    'tags': {},
    'user_id': '',
    'version': '2'}
{   'creation_timestamp': 1666855316629,
    'current_stage': 'Production',
    'description': '',
    'last_updated_timestamp': 16669387802